# load libs, system messages and api keys

In [10]:
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import time
from openai import OpenAI
import json
import pandas as pd
from transformers import LlamaTokenizer


directory_path = 'D:/projects/unity/Enlighten/Assets/Scripts'

geminiApiKey=''
openAiApiKey=''



with open('SystemMessages\codeSystemMessage.txt', 'r') as file:
    code_systemMessage = file.read()

with open('SystemMessages\NormalSystemMessage.txt', 'r') as file:
    normal_systemMessage = file.read()

with open('SystemMessages\TestQuestionsSystemMessage.txt', 'r') as file:
    test_systemMessage = file.read()


# find all file paths

In [4]:
import glob
# crawl on all files inside a directory and return all .cs files
def get_cs_files(directory):
    cs_files = glob.glob(directory + '/**/*.cs', recursive=True)
    return cs_files

cs_files = get_cs_files(directory_path)

# rremove these file-names from the list
masked_files = ['StaticTestLoader.cs','GymLevelManager.cs','PlayerDirectCameraShakeManager.cs','Movement.cs','Mimic.cs','Leg.cs','CylinderMesh.cs']
for mask in masked_files:
    cs_files = [file for file in cs_files if mask not in file]


len(cs_files)

91

# read a single file

In [5]:
with open(cs_files[3], 'r') as file:
    file_content = file.read()

print(file_content)

using System;
using UnityEngine;
using UnityEngine.AI;

/*
 * Beast.States.BeastState
 * Beast is implemented as a custom made state machine, each state is a class that inherits from BeastState
 * BeastState.cs is the base class (abstract class also Serializable too) of the beast states, it contains all the common variables and methods
 */

[Serializable]
public abstract class BeastState {
    // ChangeState() from BeastManager is always passed as changeState, so that the state can change the state of the beast
    protected Action<BeastManager.State> changeState;

    protected Transform player;
    protected Transform head;
    protected NavMeshAgent navMeshAgent;
    protected Transform beastTransform;
    protected LayerMask layerMask;
    protected float normalSpeed;


    // the state of the beast (enum)
    public abstract BeastManager.State State { get; protected set; }

    //injected dependencies
    public void Init(Transform beastTransform, Transform player, Transform head,

# count all tokens in all classes

In [ ]:

file_content =""
for doc in cs_files:
    with open(doc, 'r') as file:
        file_content = file_content + file.read()



tokenizer = LlamaTokenizer.from_pretrained("cognitivecomputations/dolphin-llama2-7b")
tokens = tokenizer.tokenize(file_content)
len(tokens)


In [6]:
tokens_series=pd.Series(tokens)
tokens_series.value_counts().head(10)

<0x0A>     6188
.          2064
▁▁▁        1651
▁▁▁▁▁▁▁    1595
;          1273
▁the        975
▁//         925
▁{          723
(           650
,           646
Name: count, dtype: int64

# open ai chat

In [ ]:
client = OpenAI(api_key=openAiApiKey)

def get_raw_text_openai(file_content,systemMessage):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    max_tokens=10000,
    messages=[
      {"role": "system", "content": systemMessage},
      {"role": "user", "content": file_content},
    ]
  )

  return response.choices[0].message.content

# gemini chat

In [6]:
genai.configure(api_key=geminiApiKey)
model = genai.GenerativeModel('gemini-pro')

def get_raw_text_gemini(file_content,systemMessage):
    response = model.generate_content(systemMessage+"\n\n"+file_content
                                    ,generation_config=genai.types.GenerationConfig(max_output_tokens=4000))
    # candidates=response.candidates[0].content.parts[0].text

    return(response.text)




# get data from a single file

In [12]:
def make_df(text):
    data = json.loads(text)
    df = pd.DataFrame(data)
    df=df.map(lambda x: x.replace('\\n', '\n'))
    return df

def peek_dataset(i):
    print(df['Q'][i])
    print(df['A'][i])


# read file content
with open(cs_files[7], 'r') as file:
    file_content = file.read()
    
# generate question and answer pairs
raw_response=get_raw_text_gemini(file_content,test_systemMessage)

# convert to dataframe
df=make_df(raw_response)


peek_dataset(2)
df



,class,Question,a,b,c,d,Answer
0,BeastStateScaping.cs,"In BeastStateScaping.cs, what is the State enu...",Eating,Patrolling,Escaping,Sleeping,c
1,BeastStateScaping.cs,"In BeastStateScaping.cs, what happens when the...",The beast runs to its hiding place,The beast changes to the sleep state,The beast becomes invisible,The beast attacks the player,b
2,BeastStateScaping.cs,"In BeastStateScaping.cs, what is the purpose o...",To set the beast's speed to fast,To set the beast's destination to its spawn po...,To change the beast's state to sleep,To make the beast look at the player,b


# entire class data

In [83]:
def path_to_className(path):
    return path.split('\\')[-1]

# randomize question template
# question_template =pd.read_csv('Datasets\question_template.csv',header=None)[0]
# import random
# random.choice(question_template).replace('{class_name}', "ClassName.cs")


df_entire_class = pd.DataFrame(columns=['class','Q','A'])
for cs_file in cs_files:
    with open(cs_file, 'r') as file:
        file_content = file.read()

    class_name=path_to_className(cs_file)
    Q = f'write the entire class of {class_name}'
    A = f'```csharp\n {file_content} \n```'
    df_entire_class = pd.concat([df_entire_class, pd.DataFrame([{'class':class_name,'Q':Q,'A':A}])], ignore_index=True)
    
df_entire_class

,class,Q,A
0,BeastAttack.cs,write the entire class of BeastAttack.cs,```csharp\n using Cinemachine;\nusing DG.Tween...
1,BeastKillsPlayerSequenceAnimator.cs,write the entire class of BeastKillsPlayerSequ...,```csharp\n using UnityEngine;\nusing UnityEng...
2,BeastManager.cs,write the entire class of BeastManager.cs,```csharp\n using System;\nusing DG.Tweening;\...
3,BeastState.cs,write the entire class of BeastState.cs,```csharp\n ﻿using System;\nusing UnityEngine;...
4,BeastStateKilling.cs,write the entire class of BeastStateKilling.cs,```csharp\n ﻿using System;\nusing UnityEngine;...
...,...,...,...
86,FullScreenEffectsManager.cs,write the entire class of FullScreenEffectsMan...,```csharp\n using System;\nusing DG.Tweening;\...
87,GamePlayCanvasManager.cs,write the entire class of GamePlayCanvasManage...,```csharp\n using UnityEngine;\n\n/*\n * Ui.Ga...
88,UIBeastManager.cs,write the entire class of UIBeastManager.cs,```csharp\n ﻿using DG.Tweening;\nusing UnityEn...
89,UICurrentObjectivesManager.cs,write the entire class of UICurrentObjectivesM...,```csharp\n using TMPro;\nusing UnityEngine;\n...


# descriptive Data

In [84]:
descriptiveDocsPath = 'D:\\projects\\unity\\Enlighten\\Assets\\Notes\\Enlighten.txt'
with open(descriptiveDocsPath, 'r') as file:
    file_content = file.read()

sections=file_content.split('#')[1:]


df_descriptive = pd.DataFrame(columns=['class','Q','A'])
for section in sections:
    first, sep, second=section.partition(":\n")
    Q = f'write about the {first} in Enlighten'
    A = second

    df_descriptive = pd.concat([df_descriptive, pd.DataFrame([{'class':"descriptive",'Q':Q,'A':A}])], ignore_index=True)
    
df_descriptive


,class,Q,A
0,descriptive,write about the Overview in Enlighten,Enlighten is an Indie game being made in Unity...
1,descriptive,write about the GamePlay in Enlighten,- game is first person - puzzle/horror\n- play...
2,descriptive,write about the Player items in Enlighten,- a flashlight [flash lights power runs out ov...
3,descriptive,write about the Puzzles in Enlighten,- puzzles are the main part of the game\n- puz...
4,descriptive,write about the Beast in Enlighten,- the beast is a monster that hunts the player...
5,descriptive,write about the Main game loop in Enlighten,- player enters a closed environment (like a b...
6,descriptive,write about the NPC in Enlighten,*Kits:\n- Kits is a cute girl with a cat mask ...
7,descriptive,write about the Scripts in Enlighten,- void Awake() : called when the script instan...


# **Generate All Data**

In [120]:
all_dataframes=list()

for i in range(len(cs_files)):
    cs_file=cs_files[i]
    with open(cs_file, 'r') as file:
        file_content = file.read()

    # non-code questions
    raw_response=get_raw_text_gemini(file_content,normal_systemMessage)
    df_normal=make_df(raw_response)
    display(df_normal)

    # code questions
    raw_response=get_raw_text_gemini(file_content,code_systemMessage)
    df_code=make_df(raw_response)
    display(df_code)

    # combine both
    new_df=pd.concat([df_normal, df_code]).reset_index(drop=True)
    all_dataframes.append(new_df)

    print(f'completed {i} of {len(cs_files)}')



df=pd.concat(all_dataframes).reset_index(drop=True)
df



,class,Q,A
0,UiHintManager.cs,what is the responsibility of the UiHintManage...,UiHintManager is responsible for the hint mess...
1,UiHintManager.cs,What happens when the UiHintManager class is e...,"When the UiHintManager class is enabled, it su..."
2,UiHintManager.cs,What happens when the UiHintManager class is d...,"When the UiHintManager class is disabled, it u..."
3,UiHintManager.cs,What does the mainHintText variable in the UiH...,The mainHintText variable is a reference to a ...
4,UiHintManager.cs,What is the purpose of the Start method in the...,The Start method is called when the UiHintMana...
5,UiHintManager.cs,What event is listened to in the UiHintManager...,The UiEventHub.OnMainHint event is listened to...
6,UiHintManager.cs,What is the purpose of the OnMainHint method i...,The OnMainHint method in the UiHintManager cla...
7,UiHintManager.cs,How does the UiHintManager class manage hints?,The UiHintManager class manages hints by subsc...
8,UiHintManager.cs,When does the UiHintManager class become active?,The UiHintManager class becomes active when it...
9,UiHintManager.cs,What happens when the UiHintManager class is d...,"When the UiHintManager class is disabled, it u..."


,class,Q,A
0,UiHintManager.cs,"In 'UiHintManager.cs', where is the object to ...",```csharp\n[SerializeField] private TextMeshP...
1,UiHintManager.cs,"In 'UiHintManager.cs', what happens when OnMai...","```csharp\nif (message == """") {\n mainHintTe..."
2,UiHintManager.cs,"In 'UiHintManager.cs', how is the hint message...",```csharp\nmainHintText.text = message;\n```
3,UiHintManager.cs,"In 'UiHintManager.cs', how is the UI element t...",```csharp\nmainHintText.transform.parent.game...
4,UiHintManager.cs,"In 'UiHintManager.cs', what event is subscribe...",```csharp\nUiEventHub.OnMainHint += OnMainHin...
5,UiHintManager.cs,"In 'UiHintManager.cs', what event is unsubscri...",```csharp\nUiEventHub.OnMainHint -= OnMainHin...
6,UiHintManager.cs,"In 'UiHintManager.cs', when is the hint UI shown?","```csharp\nif (message != """") {\n mainHintTe..."
7,UiHintManager.cs,"In 'UiHintManager.cs', where is the OnDisable ...",```csharp\nprivate void OnDisable() {\n //un...
8,UiHintManager.cs,"In 'UiHintManager.cs', what is the function of...",```csharp\nvoid OnMainHint(string message) {\...
9,UiHintManager.cs,"In 'UiHintManager.cs', where is the OnEnable m...",```csharp\nprivate void OnEnable() {\n //sub...


completed 90 of 91


,class,Q,A
0,BeastAttack.cs,What is the purpose of the BeastAttack class?,The BeastAttack class controls the beast's att...
1,BeastAttack.cs,What is the purpose of the strength field in t...,The strength field determines the force with w...
2,BeastAttack.cs,What component does the 'attackAudioClip' vari...,The attackAudioClip variable stores an AudioCl...
3,BeastAttack.cs,What does the OnTriggerEnter method do?,The OnTriggerEnter method is called when the B...
4,BeastAttack.cs,"What is the _collider field, and what is its p...",The _collider field is a reference to the Beas...
...,...,...,...
1806,UiHintManager.cs,"In 'UiHintManager.cs', what event is unsubscri...",```csharp\nUiEventHub.OnMainHint -= OnMainHin...
1807,UiHintManager.cs,"In 'UiHintManager.cs', when is the hint UI shown?","```csharp\nif (message != """") {\n mainHintTe..."
1808,UiHintManager.cs,"In 'UiHintManager.cs', where is the OnDisable ...",```csharp\nprivate void OnDisable() {\n //un...
1809,UiHintManager.cs,"In 'UiHintManager.cs', what is the function of...",```csharp\nvoid OnMainHint(string message) {\...


# concat all data and save

In [123]:
df=pd.concat([df,df_entire_class,df_descriptive]).reset_index(drop=True)
df
df.to_csv('AllData.csv', index=False)


# count all data tokens

In [4]:
all=pd.read_csv('Datasets\AllData.csv')
tokenizer = LlamaTokenizer.from_pretrained("cognitivecomputations/dolphin-llama2-7b")
tokens = tokenizer.tokenize(all.to_string())
len(tokens)

977296

# Generate Test Data

In [13]:
all_dataframes_test=list()

for i in range(len(cs_files)):
    cs_file=cs_files[i]
    with open(cs_file, 'r') as file:
        file_content = file.read()

    raw_response=get_raw_text_gemini(file_content,test_systemMessage)
    new_df=make_df(raw_response)
    display(new_df)

    all_dataframes_test.append(new_df)
    print(f'completed {i} of {len(cs_files)}')



df_test=pd.concat(all_dataframes_test).reset_index(drop=True)
df_test


,class,Question,a,b,c,d,Answer
0,BeastAttack.cs,What does the BeastAttack class do in the Unit...,Manages player movement,Generates sounds,Manages the beast attack behavior and player h...,Controls the camera,c
1,BeastAttack.cs,What does the gameObject that uses the BeastAt...,A Rigidbody,A Collider,A Cinemachine Impulse Source,All of the above,d
2,BeastAttack.cs,What happens when the player gets hit while in...,The player is pushed back,The player is pushed back and the beast enters...,The player is pushed back and the beast dies,The player is not affected,d


completed 0 of 91


,class,Question,a,b,c,d,Answer
0,BeastKillsPlayerSequenceAnimator.cs,What is the purpose of the BeastKillsPlayerSeq...,To control player movement,To manage the beast killing player animation s...,To handle player combat actions,To store references to key player components,b
1,BeastKillsPlayerSequenceAnimator.cs,"In the BeastKillsPlayerSequenceAnimator class,...",Finds the farthest destination from the player,Activates and plays the animation sequence,Returns a random destination,Sets the player position,b
2,BeastKillsPlayerSequenceAnimator.cs,What does the OnRespawnClicked method do in th...,Deactivates the animation sequence,Teleports the player,Sets the player position,Finds the farthest destination from the player,a


completed 1 of 91


,class,Question,a,b,c,d,Answer
0,Beast.BeastManager.cs,"What states can the beast be in, according to ...","Sleep, On Target, Roaming, Escaping, and Bleeding","Sleep, On Target, Roaming, Escaping, and Killing","Sleep, On Target, Roaming, Escaping, and Attac...","Sleep, On Target, Roaming, Escaping, and Stunned",b
1,Beast.BeastManager.cs,What happens when the lights are back on? (eve...,The beast changes to state Sleep.,The beast changes to state Killing.,The beast changes to state Escaping.,The beast changes to state Roaming.,c
2,Beast.BeastManager.cs,What is the purpose of the _scapingState varia...,To control the beast's movement when it is esc...,To control the beast's movement when it is sle...,To control the beast's movement when it is roa...,To control the beast's movement when it is on ...,a


...

,class,Question,a,b,c,d,Answer
0,UICurrentObjectivesManager.cs,What does the UICurrentObjectivesManager class...,Player movement,Enemy behaviour,Story progression,Current objectives messages,d
1,UICurrentObjectivesManager.cs,What is the responsibility of the CurrentObjec...,Updating the current objective UI element,Hiding the UI element,Starting the game,Unsubscribing from events,a
2,UICurrentObjectivesManager.cs,Which event does the UICurrentObjectivesManage...,UiEventHub.OnGameStart,UiEventHub.OnEnemyKilled,UiEventHub.OnPlayerDeath,UiEventHub.OnCurrentObjectiveUpdate,d


completed 89 of 91


,class,Question,a,b,c,d,Answer
0,UiHintManager.cs,What does UiHintManager do?,Displays hints to the player.,Manages player inventory.,Controls player movement.,Calculates the player's score.,a
1,UiHintManager.cs,What happens when OnMainHint is called?,The main hint text is updated.,The player's score is increased.,The game is paused.,The player's health is restored.,a
2,UiHintManager.cs,What is the purpose of the mainHintText field?,To store the player's name.,To store the current level.,To display hints to the player.,To store the game's high score.,c


completed 90 of 91


,class,Question,a,b,c,d,Answer
0,BeastAttack.cs,What does the BeastAttack class do in the Unit...,Manages player movement,Generates sounds,Manages the beast attack behavior and player h...,Controls the camera,c
1,BeastAttack.cs,What does the gameObject that uses the BeastAt...,A Rigidbody,A Collider,A Cinemachine Impulse Source,All of the above,d
2,BeastAttack.cs,What happens when the player gets hit while in...,The player is pushed back,The player is pushed back and the beast enters...,The player is pushed back and the beast dies,The player is not affected,d
3,BeastKillsPlayerSequenceAnimator.cs,What is the purpose of the BeastKillsPlayerSeq...,To control player movement,To manage the beast killing player animation s...,To handle player combat actions,To store references to key player components,b
4,BeastKillsPlayerSequenceAnimator.cs,"In the BeastKillsPlayerSequenceAnimator class,...",Finds the farthest destination from the player,Activates and plays the animation sequence,Returns a random destination,Sets the player position,b
...,...,...,...,...,...,...,...
266,UICurrentObjectivesManager.cs,What is the responsibility of the CurrentObjec...,Updating the current objective UI element,Hiding the UI element,Starting the game,Unsubscribing from events,a
267,UICurrentObjectivesManager.cs,Which event does the UICurrentObjectivesManage...,UiEventHub.OnGameStart,UiEventHub.OnEnemyKilled,UiEventHub.OnPlayerDeath,UiEventHub.OnCurrentObjectiveUpdate,d
268,UiHintManager.cs,What does UiHintManager do?,Displays hints to the player.,Manages player inventory.,Controls player movement.,Calculates the player's score.,a
269,UiHintManager.cs,What happens when OnMainHint is called?,The main hint text is updated.,The player's score is increased.,The game is paused.,The player's health is restored.,a


In [14]:
df_test.to_csv('TestData.csv', index=False)